# Parte 1: Recolección de links
### Web Scraping a "Semana" para investigación referente a los cambios climáticos en Colombia

### Paso 1: Importación de variables

In [1]:
import pandas as pd
import time
import random
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from random import shuffle

### Paso 2: Definición de opciones para el navegador

In [2]:
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument('--window-size=1420,1080')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--remote-debugging-port=9222")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_experimental_option("excludeSwitches", ["disable-popup-blocking"])

### Paso 3: Definición de clase personalizada

In [3]:
# Clase para personalización
class bcolors:
    WARNING = '\033[93m'
    FAIL = '\033[91m'

# Control de errores
try:   
    # Definición de función
    def Seconds(result):
        hours = round(result/3600)
        minutes = round((result - (hours * 3600))/60)
        seconds = round(result - ((hours * 3600) + (minutes * 60)))
        if seconds < 0: seconds = seconds*-1
        if minutes < 0: minutes = minutes*-1
        final_time = "El sitema tardó: " + str(hours) + " horas " + str(minutes) + " minutos " + str(seconds) + " segundos en finalizar su ejeción"
        return final_time
    
except Exception as e: 
    print(bcolors.FAIL + "Revisa el error => ", e) # Impresión del error personalizado

### Paso 4: Estructuración del WebScraping para captura de links

In [4]:
# PASO 4.1: Definición de variables para captura de información -----------------------------------------------------------------------#
headers = []
headers_text = []
headers_link = []
all_news = []

# PASO 4.1.1 Definición de frases de interés a consultar (serviran para iteración en ciclo) -------------------------------------------#
phrases_of_interest = [
    'Siembra de árboles',
    'Reciclaje',
    'Economía circular',
    'Matriz energética',
    'Energía limpia',
    'Energía renovable',
    'Energías alternativas',
    'Energías verdes',
    'Derecho ambiental',
    'Regulación ambiental',
    'Normativa ambiental',
    'Convención Marco de las Naciones Unidas sobre Cambio Climático,',
    'Acuerdo de París',
    'Protocolo de Tokio',
    'Transición energética',
    'Sostenibilidad ambiental',
    'Refugiado climático ',
    'Estrategia 2050'
]

In [5]:
# PASO 4.2: Aviso para el usuario final de que se inicio el web-scraping --------------------------------------------------------------#
print("Se inició el Web Scraping a 'Semana', por favor espere...") # Mensaje para visualización del inicio del proceso

# 4.2.1 Variable para definir el inicio de tiempo en ejecución
start = time.time()

Se inició el Web Scraping a 'Semana', por favor espere...


In [6]:
# PASO 4.3: Estructura base para acceder a la página en cuestión ----------------------------------------------------------------------#
service = Service(executable_path=r'../chromedriver-win64/chromedriver.exe') # Creación del servicio mediante el driver
options = webdriver.ChromeOptions() # Definición de variable con las opciones para chrome
driver = webdriver.Chrome(service=service, options=options) # Creación del objeto del WebDriver

# 4.3.1 Acceso a la página en cuestión
driver.get("https://www.semana.com/") # Página principal
time.sleep(random.randrange(1,5)) # Simulación de comportamiento humano

In [7]:
# PASO 4.4: Estructura base para acceder a la página en cuestión ----------------------------------------------------------------------#
button_xpath = '//*[@id="fusion-app"]/div/header/div[3]/div[4]/div/div[1]/div[1]/div[1]/div[1]/div/label' # xpath del botón
menu_button = driver.find_element(By.XPATH, button_xpath) # Botón de lupa (sidebar)
menu_button.click(); # Acción de click en el botón
time.sleep(random.randrange(3,5)) # Simulación de comportamiento humano

In [8]:
# PASO 4.5: Ubicación y selección del campo para búsqueda -----------------------------------------------------------------------------#
search_bar = driver.find_element(By.XPATH, '//*[@id="queryly_query"]') # Campo de búsqueda (ventana emergente)

In [9]:
# PASO 4.6: Ciclo para iterar sobre las frases de interés en el campo para búsqueda ---------------------------------------------------#
for phrase in phrases_of_interest:
    search_bar.clear() # Limpieza del campo
    search_bar.send_keys(phrase) # Envío de la frase de interés al campo de búsqueda
    time.sleep(random.randrange(3,5)) # Simulación de comportamiento humano
    search_bar.send_keys(Keys.RETURN) # Envío de consulta (Enter)    
    time.sleep(random.randrange(3,5)) # Simulación de comportamiento humano

    # 4.6.1 Control de excepción en caso dado que el filtro de "Medio ambiente" no esté presente en la página
    try:
        # Si está presente el filtro se aplica
        css_selector = 'div.filter_item[data-filter-value="Medio ambiente"] a' # Definición del css del filtro
        filter_button = driver.find_element(By.CSS_SELECTOR, css_selector) # Intento de encontrar el botón para aplicar el filtro
        filter_button.click() # Acción de click en el botón
    except NoSuchElementException:
        pass # Si NO está presente el filtro NO se aplica y la iteración continua
    
    time.sleep(random.randrange(3,5)) # Simulación de comportamiento humano

    # 4.6.2 identificación de la cantida de resultados encontrados
    results = driver.find_element(By.XPATH, '//*[@id="resultdata"]')

    time.sleep(random.randrange(3,5)) # Simulación de comportamiento humano

    # 4.6.3 Ciclo While para recorrer todas las páginas
    while(True):
        
        time.sleep(random.randrange(3,5)) # Simulación de comportamiento humano
        
        # 4.6.4 Identificación del Div que contiene las noticias en consulta
        elements = driver.find_elements(By.CSS_SELECTOR, "div.queryly_item_row")
        
        # 4.6.5 Ciclo for para iteración de todos los elementos encontrados (los elementos son las noticias)
        for element in elements:
        # ****************************************************************************************************************
            ## NOTA: las etiquetas (a) representan los links en HTML
            
            # 4.6.5.1 Identificación de la etiqueta (a) en los elementos
            headers_tag_a =  element.find_elements(By.TAG_NAME, 'a')
            # 4.6.5.2 (parte 1) Condicional para validación de "encontrada" la etiqueta (a)
            if headers_tag_a:
                # 4.6.5.3 Ciclo for para iterar sobre comulo de etiquetas encontradas para extraer información
                for header in headers_tag_a:
                    headers_text.append(header.text) # Extracción de los títulos de las noticias
                    headers_link.append(header.get_attribute('href')) # Extracción de los links de la noticias en su atributo "href"
            # 4.6.5.2 (parte 2) Dado el caso que en la iteración no se encuentre la etiqueta se llenan los campos con puntos suspensivos (...)
            else: 
                headers_text.append('...') # Estandar de llenado en caso de no encontrar resultados
                headers_link.append('...') # Estandar de llenado en caso de no encontrar resultados
        # ****************************************************************************************************************

        
        # 4.6.6 Construcción del diccionario 
        news = {
            "headers_text": headers_text, # Título de la noticia
            "headers_link": headers_link, # Link de la noticia
        }

        # 4.6.7 Construcción inicial del dataframe
        df = pd.DataFrame(news)

        # (Opcional) Modo DEBUG, sirve para validar como el DataFrame se está llenando según las iteraciones
        display(df)

        time.sleep(random.randrange(3,5)) # Simulación de comportamiento humano

        # 4.6.8 Control para el comportamiento al no encontrar el botón de siguiente
        try:

            # 4.6.8.1 Búsqueda del botón "Siguiente"
            next_button = driver.find_element(By.CSS_SELECTOR, 'a.next_btn') # Botón "Siguiente"
            # Script para movilizarnos a la parte inferior de la página para mayor facilidad de encontrar el botón
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button) # script y ejecución del mismo

            # 4.6.8.2 (parte 1) Condicional para la validación de existencia del elemento
            if next_button: 
                # 4.6.8.2.1 Limpieza del array que contiene las noticias anteriores (sirve para que no se dupliquen en el DataFrame)
                elements = []
                time.sleep(random.randrange(3,5)) # Simulación de comportamiento humano
                next_button.click() # Acción de click en el botón
            # 4.6.8.2 (parte 2) En caso de no encontrar el botón hay 2 posibilidades
                # 1. La consulta solo tiene 1 página
                # 2. Ya no hay más páginas por iterar
            else: break; 
        except Exception as e:
            break # Si (NO) encuentra el botón de siguiente cierra el ciclo y continua en el ciclo padre
        except NoSuchElementException:
            break # Si (NO) encuentra el botón de siguiente cierra el ciclo y continua en el ciclo padre
        except TimeoutException:
            break # Si (NO) encuentra el botón de siguiente cierra el ciclo y continua en el ciclo padre

    # 4.6.9 Repetición del proceso de búsqueda
    button_xpath = '//*[@id="fusion-app"]/div/header/div[2]/div[3]/div/div[1]/div[1]/div[1]/div[1]/div/label' # xpath del botón
    menu_button = driver.find_element(By.XPATH, button_xpath) # Botón de lupa (sidebar)
    menu_button.click(); # Acción de click en el botón
    time.sleep(random.randrange(3,5)) # Simulación de comportamiento humano
    search_bar = driver.find_element(By.XPATH, '//*[@id="queryly_query"]') # Campo de búsqueda (ventana emergente)
    time.sleep(random.randrange(3,5)) # Simulación de comportamiento humano

,headers_text,headers_link
0,La CVC sembró el árbol de mangle número 35 mil...,https://www.semana.com/sostenibilidad/medio-am...
1,Con la siembra de 150.000 árboles reforestarán...,https://www.semana.com/sostenible/medio-ambien...
2,Textilera colombiana Lafayette sembrará 8.000 ...,https://www.semana.com/sostenibilidad/medio-am...
3,Se sembraron 10 mil árboles nativos para resta...,https://www.semana.com/medio-ambiente/articulo...
4,Inician la siembra de un millón de árboles en ...,https://www.semana.com/sostenible/medio-ambien...
5,Bogotá reverdece con más de 740 árboles sembra...,https://www.semana.com/sostenible/medio-ambien...
6,Estos son los cinco departamentos que lideran ...,https://www.semana.com/sostenible/medio-ambien...
7,Estos son los 10 municipios que más han sembra...,https://www.semana.com/sostenible/medio-ambien...
8,La CAR sembrará más de 30.000 árboles en la ro...,https://www.semana.com/sostenible/medio-ambien...
9,Familias del Meta recibieron 12.000 árboles pa...,https://www.semana.com/sostenibilidad/medio-am...


,headers_text,headers_link
0,La CVC sembró el árbol de mangle número 35 mil...,https://www.semana.com/sostenibilidad/medio-am...
1,Con la siembra de 150.000 árboles reforestarán...,https://www.semana.com/sostenible/medio-ambien...
2,Textilera colombiana Lafayette sembrará 8.000 ...,https://www.semana.com/sostenibilidad/medio-am...
3,Se sembraron 10 mil árboles nativos para resta...,https://www.semana.com/medio-ambiente/articulo...
4,Inician la siembra de un millón de árboles en ...,https://www.semana.com/sostenible/medio-ambien...
5,Bogotá reverdece con más de 740 árboles sembra...,https://www.semana.com/sostenible/medio-ambien...
6,Estos son los cinco departamentos que lideran ...,https://www.semana.com/sostenible/medio-ambien...
7,Estos son los 10 municipios que más han sembra...,https://www.semana.com/sostenible/medio-ambien...
8,La CAR sembrará más de 30.000 árboles en la ro...,https://www.semana.com/sostenible/medio-ambien...
9,Familias del Meta recibieron 12.000 árboles pa...,https://www.semana.com/sostenibilidad/medio-am...


,headers_text,headers_link
0,La CVC sembró el árbol de mangle número 35 mil...,https://www.semana.com/sostenibilidad/medio-am...
1,Con la siembra de 150.000 árboles reforestarán...,https://www.semana.com/sostenible/medio-ambien...
2,Textilera colombiana Lafayette sembrará 8.000 ...,https://www.semana.com/sostenibilidad/medio-am...
3,Se sembraron 10 mil árboles nativos para resta...,https://www.semana.com/medio-ambiente/articulo...
4,Inician la siembra de un millón de árboles en ...,https://www.semana.com/sostenible/medio-ambien...
5,Bogotá reverdece con más de 740 árboles sembra...,https://www.semana.com/sostenible/medio-ambien...
6,Estos son los cinco departamentos que lideran ...,https://www.semana.com/sostenible/medio-ambien...
7,Estos son los 10 municipios que más han sembra...,https://www.semana.com/sostenible/medio-ambien...
8,La CAR sembrará más de 30.000 árboles en la ro...,https://www.semana.com/sostenible/medio-ambien...
9,Familias del Meta recibieron 12.000 árboles pa...,https://www.semana.com/sostenibilidad/medio-am...


,headers_text,headers_link
0,La CVC sembró el árbol de mangle número 35 mil...,https://www.semana.com/sostenibilidad/medio-am...
1,Con la siembra de 150.000 árboles reforestarán...,https://www.semana.com/sostenible/medio-ambien...
2,Textilera colombiana Lafayette sembrará 8.000 ...,https://www.semana.com/sostenibilidad/medio-am...
3,Se sembraron 10 mil árboles nativos para resta...,https://www.semana.com/medio-ambiente/articulo...
4,Inician la siembra de un millón de árboles en ...,https://www.semana.com/sostenible/medio-ambien...
...,...,...
75,"El Río Bogotá avanza en su recuperación\nlun, ...",https://www.semana.com/sostenible/medio-ambien...
76,Acueducto de Bogotá invirtió más 2 billones en...,https://www.semana.com/sostenible/medio-ambien...
77,Colombia ha recuperado más de 22 mil hectáreas...,https://www.semana.com/medio-ambiente/articulo...
78,¿Qué retos tiene el río Bogotá para 2022?\nlun...,https://www.semana.com/sostenible/medio-ambien...


,headers_text,headers_link
0,La CVC sembró el árbol de mangle número 35 mil...,https://www.semana.com/sostenibilidad/medio-am...
1,Con la siembra de 150.000 árboles reforestarán...,https://www.semana.com/sostenible/medio-ambien...
2,Textilera colombiana Lafayette sembrará 8.000 ...,https://www.semana.com/sostenibilidad/medio-am...
3,Se sembraron 10 mil árboles nativos para resta...,https://www.semana.com/medio-ambiente/articulo...
4,Inician la siembra de un millón de árboles en ...,https://www.semana.com/sostenible/medio-ambien...
...,...,...
82,"Cambiar la dieta del ganado, alternativa para ...",https://www.semana.com/cambiar-la-dieta-del-ga...
83,"Pacto contra el tráfico ilegal de madera\nmié,...",https://www.semana.com/pacto-contra-trafico-il...
84,"Guaviare, tierra de nadie\nmar, 26 jun 2018\nJ...",https://www.semana.com/deforestacion-en-guavia...
85,La 'incomprendida' medida ambiental del alcald...,https://www.semana.com/alcalde-de-cali-maurici...


...

,headers_text,headers_link
0,La CVC sembró el árbol de mangle número 35 mil...,https://www.semana.com/sostenibilidad/medio-am...
1,Con la siembra de 150.000 árboles reforestarán...,https://www.semana.com/sostenible/medio-ambien...
2,Textilera colombiana Lafayette sembrará 8.000 ...,https://www.semana.com/sostenibilidad/medio-am...
3,Se sembraron 10 mil árboles nativos para resta...,https://www.semana.com/medio-ambiente/articulo...
4,Inician la siembra de un millón de árboles en ...,https://www.semana.com/sostenible/medio-ambien...
...,...,...
1415,“Programas REDD+ deben incluir a quienes histó...,https://www.semana.com/redd-deben-incluir-a-qu...
1416,Cinco ciudades lideran la lucha contra el camb...,https://www.semana.com/ciudades-que-lideran-la...
1417,‘Aviturismo’ en Perú: mirarlas en vez de matar...,https://www.semana.com/aviturismo-en-peru-mira...
1418,Cinco nuevos saltamontes descubiertos en las s...,https://www.semana.com/medio-ambiente/articulo...


,headers_text,headers_link
0,La CVC sembró el árbol de mangle número 35 mil...,https://www.semana.com/sostenibilidad/medio-am...
1,Con la siembra de 150.000 árboles reforestarán...,https://www.semana.com/sostenible/medio-ambien...
2,Textilera colombiana Lafayette sembrará 8.000 ...,https://www.semana.com/sostenibilidad/medio-am...
3,Se sembraron 10 mil árboles nativos para resta...,https://www.semana.com/medio-ambiente/articulo...
4,Inician la siembra de un millón de árboles en ...,https://www.semana.com/sostenible/medio-ambien...
...,...,...
1425,“La actividad extractiva de minerales como se ...,https://www.semana.com/portada-semana-sostenib...
1426,"¿Se acabará el agua en Colombia?\ndom, 25 abr ...",https://www.semana.com/nacion/medio-ambiente/a...
1427,"Pacto contra el tráfico ilegal de madera\nmié,...",https://www.semana.com/pacto-contra-trafico-il...
1428,Humedales de Colombia continúan en ‘cuidados i...,https://www.semana.com/humedales-de-colombia-c...


,headers_text,headers_link
0,La CVC sembró el árbol de mangle número 35 mil...,https://www.semana.com/sostenibilidad/medio-am...
1,Con la siembra de 150.000 árboles reforestarán...,https://www.semana.com/sostenible/medio-ambien...
2,Textilera colombiana Lafayette sembrará 8.000 ...,https://www.semana.com/sostenibilidad/medio-am...
3,Se sembraron 10 mil árboles nativos para resta...,https://www.semana.com/medio-ambiente/articulo...
4,Inician la siembra de un millón de árboles en ...,https://www.semana.com/sostenible/medio-ambien...
...,...,...
1435,La CVC y la Agencia Francesa firmaron convenio...,https://www.semana.com/sostenibilidad/medio-am...
1436,Providencia inicia piloto de cultivo de corale...,https://www.semana.com/sostenible/medio-ambien...
1437,"Armenia, Pasto y Yopal también quieren convert...",https://www.semana.com/sostenible/medio-ambien...
1438,El 40% de las especies del mundo habitan en lo...,https://www.semana.com/el-40-de-las-especies-d...


,headers_text,headers_link
0,La CVC sembró el árbol de mangle número 35 mil...,https://www.semana.com/sostenibilidad/medio-am...
1,Con la siembra de 150.000 árboles reforestarán...,https://www.semana.com/sostenible/medio-ambien...
2,Textilera colombiana Lafayette sembrará 8.000 ...,https://www.semana.com/sostenibilidad/medio-am...
3,Se sembraron 10 mil árboles nativos para resta...,https://www.semana.com/medio-ambiente/articulo...
4,Inician la siembra de un millón de árboles en ...,https://www.semana.com/sostenible/medio-ambien...
...,...,...
1455,Declaración de la Naturaleza Urbana: ¿qué es y...,https://www.semana.com/sostenible/medio-ambien...
1456,Estos proyectos ambientales esperan debate en ...,https://www.semana.com/sostenible/medio-ambien...
1457,¿Cuáles son los países que avanzan realmente h...,https://www.semana.com/cuales-son-los-paises-q...
1458,El cambio climático preocupa a los colombianos...,https://www.semana.com/medio-ambiente/articulo...


,headers_text,headers_link
0,La CVC sembró el árbol de mangle número 35 mil...,https://www.semana.com/sostenibilidad/medio-am...
1,Con la siembra de 150.000 árboles reforestarán...,https://www.semana.com/sostenible/medio-ambien...
2,Textilera colombiana Lafayette sembrará 8.000 ...,https://www.semana.com/sostenibilidad/medio-am...
3,Se sembraron 10 mil árboles nativos para resta...,https://www.semana.com/medio-ambiente/articulo...
4,Inician la siembra de un millón de árboles en ...,https://www.semana.com/sostenible/medio-ambien...
...,...,...
1462,¿Qué respiran y de dónde proviene la contamina...,https://www.semana.com/medio-ambiente/articulo...
1463,La milagrosa supervivencia de los corales de V...,https://www.semana.com/medio-ambiente/articulo...
1464,¿Qué hay detrás del “veto” al carbón colombian...,https://www.semana.com/drummond-y-prodeco-veta...
1465,Cinco ciudades lideran la lucha contra el camb...,https://www.semana.com/ciudades-que-lideran-la...


In [10]:
# PASO 4.7: Cerrado del navegador (finalización del scraping)
driver.quit()

In [11]:
# PASO 4.8: Limpieza del DataFrame
df = df.drop_duplicates(subset=["headers_link"])

In [12]:
# PASO 4.9: Generación del archivo .csv con el consolidado de noticias
df.to_csv("df_news_2.csv")

# 4.9.1 Variable para definir el final de tiempo en ejecución
end = time.time()

# 4.9.2 Variable para calcular tiempo y para enviar a la función
result = end-start

# 4.9.3 Impresión del tiempo que tardó en ejecución
print(Seconds(result))

El sitema tardó: 0 horas 22 minutos 20 segundos en finalizar su ejeción


In [13]:
# PASO 5: Implresión del DataFrame consolidado y tratado
df

,headers_text,headers_link
0,La CVC sembró el árbol de mangle número 35 mil...,https://www.semana.com/sostenibilidad/medio-am...
1,Con la siembra de 150.000 árboles reforestarán...,https://www.semana.com/sostenible/medio-ambien...
2,Textilera colombiana Lafayette sembrará 8.000 ...,https://www.semana.com/sostenibilidad/medio-am...
3,Se sembraron 10 mil árboles nativos para resta...,https://www.semana.com/medio-ambiente/articulo...
4,Inician la siembra de un millón de árboles en ...,https://www.semana.com/sostenible/medio-ambien...
...,...,...
1442,Lecciones del coronavirus en tiempos de cambio...,https://www.semana.com/medio-ambiente/articulo...
1447,Neutralidad del carbono: ¿qué se requiere real...,https://www.semana.com/medio-ambiente/articulo...
1457,¿Cuáles son los países que avanzan realmente h...,https://www.semana.com/cuales-son-los-paises-q...
1458,El cambio climático preocupa a los colombianos...,https://www.semana.com/medio-ambiente/articulo...
